In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [43]:
with open('/Users/andrewpeters/GitHub/fpl/data/processed/X.npy', 'rb') as file:
    X = np.load(file)
with open('/Users/andrewpeters/GitHub/fpl/data/processed/y.npy', 'rb') as file:
    y = np.load(file)

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [48]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [49]:
dtr = DecisionTreeRegressor(max_depth = 6) #validated in my notebook that test_error is lowest at depth of ~6
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=6)

In [50]:
y_predicted = dtr.predict(X_test)

In [51]:
from helpers import score_model

In [52]:
results_df = score_model(predictions = y_predicted, y_test = y_test)

Overall Score is: 2.9714152547579773
MAE over the entire dataset is: 1.807039387987451
MAE when a player earns more than 5pts is: 3.5536031881432404


In [53]:
with open ('/Users/andrewpeters/GitHub/fpl/models/dtr_297.pkl', 'wb') as file: #filename = model + score
    pickle.dump(dtr, file)